In [50]:

import numpy as np
import random
import math
import copy

# Task 1: Generate 40 trucks with capacity between 500 to 1000Kg
trucks_capacity = np.random.randint(500, 1001, size=40)

# Task 2: Generate 120 packages with weight (between 200-1000) and distance (between 20 to 100)
packages = np.random.randint(200, 1001, size=(120, 2))
packages[:, 1] = np.random.randint(20, 101, size=120)

# Task 3: Initial assignment
def initial_assignment(trucks_capacity, packages):
    assignments = np.full((40, 3), -1, dtype=int)

    package_assigned = np.zeros(len(packages), dtype=bool)
    for t_idx, capacity in enumerate(trucks_capacity):
        for i in range(3):  # A truck can carry up to 3 packages
            # Find packages that can be carried by the current truck and are not yet assigned
            available_packages = np.argwhere((packages[:, 0] <= capacity) & (~package_assigned)).flatten()
            if len(available_packages) > 0:
                p_idx = np.random.choice(available_packages)
                assignments[t_idx][i] = p_idx
                package_assigned[p_idx] = True
            else:
                break

    return assignments, package_assigned

# Task 4: Calculate the cost
def calculate_cost(assignments, initial_capacity, packages):
    cost = 0
    delivered_packages = np.zeros(len(packages), dtype=bool)
    for t_idx, deliveries in enumerate(assignments):
        for p_idx in deliveries:
            if p_idx != -1:
                p = packages[p_idx]
                cost += (initial_capacity[t_idx] - p[0]) / p[1]
                delivered_packages[p_idx] = True

    # Add penalty for undelivered packages
    penalty = 100 * np.sum(np.logical_not(delivered_packages))
    cost += penalty

    return cost, delivered_packages

# Task 5 and 6: Use SGA optimization
def sga_optimization(assignments, trucks_capacity, packages, initial_cost, max_iter=7500, tol=0.5):
    initial_capacity = trucks_capacity.copy()
    current_cost = initial_cost
    current_assignments = copy.deepcopy(assignments)

    for iteration in range(max_iter):
        # Swap two random packages
        t1, t2 = random.sample(range(40), 2)
        p1, p2 = random.sample(range(3), 2)
        new_assignments = copy.deepcopy(current_assignments)
        new_assignments[t1][p1], new_assignments[t2][p2] = new_assignments[t2][p2], new_assignments[t1][p1]

        # Check if the swap exceeds the truck capacity
        p1_weight = packages[new_assignments[t1][p1]][0] if new_assignments[t1][p1] != -1 else 0
        p2_weight = packages[new_assignments[t2][p2]][0] if new_assignments[t2][p2] != -1 else 0
        if p1_weight > initial_capacity[t1] or p2_weight > initial_capacity[t2]:
            continue

        # Calculate new cost
        new_cost, delivered_packages = calculate_cost(new_assignments, initial_capacity, packages)

        # Check if the cost is better
        if new_cost < current_cost:
            print(f"Iteration {iteration}: Switched package {p1} of truck {t1} with package {p2} of truck {t2}, old cost: {current_cost}, new cost: {new_cost}")
            # Check for convergence
            if abs(new_cost - current_cost) < tol and np.all(delivered_packages):
                break
            current_assignments, current_cost = new_assignments, new_cost

        # Assign unassigned packages
        unassigned_packages = np.argwhere(delivered_packages == False).flatten()
        for p_idx in unassigned_packages:
            p_weight = packages[p_idx][0]
            available_trucks = np.argwhere(initial_capacity >= p_weight).flatten()
            if len(available_trucks) > 0:
                t_idx = np.random.choice(available_trucks)
                for i, delivery in enumerate(current_assignments[t_idx]):
                    if delivery == -1:
                        current_assignments[t_idx][i] = p_idx
                        delivered_packages[p_idx] = True

                        # Recalculate the cost after assigning the package
                        current_cost, _ = calculate_cost(current_assignments, initial_capacity, packages)
                        break

        # Update truck capacities
        for t_idx in range(40):
            assigned_packages = current_assignments[t_idx]
            assigned_weights = np.array([packages[p_idx][0] if p_idx != -1 else 0 for p_idx in assigned_packages])

    return current_assignments, current_cost, initial_capacity

# Run the optimization
assignments, remaining_capacity = initial_assignment(trucks_capacity, packages)
initial_cost, _ = calculate_cost(assignments, trucks_capacity, packages)
print("Initial cost:", initial_cost)

optimized_assignments, optimized_cost,initial_capacity = sga_optimization(assignments, trucks_capacity, packages, initial_cost)
print("Optimized cost:", optimized_cost)

# Print truck deliveries
print("\nTruck deliveries after optimization:")

for truck_idx, assigned_packages in enumerate(optimized_assignments):
    truck_weight = initial_capacity[truck_idx]
    assigned_weights = [packages[p_idx][0] if p_idx != -1 else 0 for p_idx in assigned_packages]

    print(f"Truck {truck_idx}: weight = {truck_weight}, package weights = {assigned_weights}")

# Find and print unassigned packages' weights
_, delivered_packages = calculate_cost(optimized_assignments, initial_capacity, packages)
unassigned_packages = np.argwhere(delivered_packages == False).flatten()
unassigned_weights = [packages[p_idx][0] for p_idx in unassigned_packages]
print("\nUnassigned packages' weights:", unassigned_weights)

Initial cost: 1777.3544116080152
Iteration 2: Switched package 2 of truck 8 with package 1 of truck 35, old cost: 1777.3544116080152, new cost: 1776.0802180596281
Iteration 4: Switched package 1 of truck 7 with package 0 of truck 39, old cost: 1776.0802180596281, new cost: 1775.143205072615
Iteration 9: Switched package 2 of truck 12 with package 0 of truck 20, old cost: 1775.143205072615, new cost: 1774.6223717392818
Iteration 10: Switched package 2 of truck 13 with package 1 of truck 4, old cost: 1774.6223717392818, new cost: 1771.143583860494
Iteration 15: Switched package 1 of truck 29 with package 2 of truck 8, old cost: 1771.143583860494, new cost: 1766.9150124319226
Iteration 18: Switched package 2 of truck 12 with package 0 of truck 9, old cost: 1668.061353895337, new cost: 1667.61145871714
Iteration 20: Switched package 0 of truck 25 with package 1 of truck 23, old cost: 1667.61145871714, new cost: 1666.9673909205299
Iteration 26: Switched package 2 of truck 35 with package 0 